## 지도학습  

### 분류 - 이진분류 
- 목표변수가 2개의 범주인 것 

In [1]:
import pandas as pd

In [2]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


### 전처리 과정

In [3]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df_TFD_extract = df_TFD[['Survived', 'Pclass', 'Age']]
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [5]:
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### Scaling & Encoding 
- split 하기 전에 실시하는게 유리 

##### Encoding with One Hot Encoding 
- 범주형 데이터를 연속형 숫자 데이터로 변환
- 범주형 항목만큼 벡터화


In [6]:
df_TFD_extract_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
oneHotEncoder = OneHotEncoder() #인스턴스화 
oneHotEncoder.fit(df_TFD_extract_preprocess[['Pclass']]) ## value값만 넣어줘야 함 [[]]을 통해 value만 넣어줌
# 해당 항목 학습한 것임.

## .value_counts() 항목만 넣어줘도 됨 / 몇개의 항목이 있는지 모를경우 전체의 값을 집어넣음

OneHotEncoder()

In [9]:
column_name = oneHotEncoder.categories_  # 컬럼 name

In [10]:
# oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray() #values # 실제 값 확인용 
encoded_data = oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()

In [11]:
df_TFD_extract_preprocess[['Pclass']][:3]

,Pclass
0,3
1,1
2,3


In [12]:
## DataFrame에 담기 위한 작업 
df_encoded_data = pd.DataFrame(data=encoded_data, columns=column_name)
df_encoded_data[:2]

,1,2,3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [13]:
df_encoded_data.index, df_encoded_data.shape

## 인덱스의 갯수가 안맞음 

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [14]:
df_TFD_extract_preprocess.index, df_TFD_extract_preprocess.shape

(Int64Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,
             ...
             880, 881, 882, 883, 884, 885, 886, 887, 889, 890],
            dtype='int64', length=714),
 (714, 3))

#### concat

In [15]:
df_TFD_extract_preprocess['Pclass'].shape

(714,)

In [16]:
# df_encoded_data = pd.get_dummies(df_TFD_extract_preprocess['Pclass'], prefix='Pclass')  ## prefix='Pclass' 컬럼명을 부여

In [17]:
df_TFD_extract_preprocess = pd.concat([df_TFD_extract_preprocess, df_encoded_data], axis=1)
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age,"(1,)","(2,)","(3,)"
0,0.0,3.0,22.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,0.0


In [18]:
## 원래 df에 새로 생성한 df를 붙이는 과정 
pd.concat([df_TFD_extract_preprocess, df_encoded_data], axis=1, ignore_index=True)

,0,1,2,3,4,5,6,7,8
0,0.0,3.0,22.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,3.0,35.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
680,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0
692,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0
697,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0
709,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0


#### Scaling - MinMaxScaler
- 목표변수는 해당사항 아님 

In [19]:
target = df_TFD_extract_preprocess['Survived']

In [20]:
features = df_TFD_extract_preprocess.drop(columns=['Survived', 'Pclass'])

In [21]:
features.columns

Index(['Age', (1,), (2,), (3,)], dtype='object')

In [22]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
# 대상이 같을때 fit_transform를 동시에 
minMaxScaler = MinMaxScaler() #인스턴스화
features = minMaxScaler.fit_transform(features)
features.shape
# 가장큰게 1, 작은게 0

TypeError: Feature names are only supported if all input features have string names, but your input has ['str', 'tuple'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

#### 정형화 단계 

In [ ]:
from sklearn.model_selection import train_test_split
# target = df_TFD_extract_preprocess['Survived']
# features = df_TFD_extract_preprocess.drop(columns=['Survived'])

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111) #random_state=111 랜덤값으로 설정
features_train.shape, target_train.shape, features_test.shape, target_test.shape

## 총 4개의 결과값 / train은 학습, test 시험, 평가
# 데이터가 500개가 넘지 않으면 split을 하지 않고 데이터를 더 모아야함

In [ ]:
target_train = df_TFD_extract_preprocess['Survived']
features_train = df_TFD_extract_preprocess.drop(columns=['Survived'])
target_train.shape, features_train.shape

### 모델 학습

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() ## 인스턴스화 
model.fit(features_train, target_train)

In [ ]:
model.coef_, model.intercept_

### 예측

In [ ]:
df_TFD_extract_preprocess[10:15]

In [ ]:
model.predict(features_train[10:15])

## 실제값 : 1,0,0,0,1 / 결과값 : [0, 0, 0, 0, 0]

In [ ]:
model.predict_proba(features_train[10:15]) ## predict_proba : 확률값

## 0일 확률 : 0.52507531 / 1일 확률 : 0.47492469

### 평가 
- accuracy_score(정확도)의 값이 0.85보다 높아야 데이터의 의미가 있음
    - 0.85 이하는 모델자체가 성능이 떨어진다는 의미 

In [ ]:
target_train_predict = model.predict(features_train) # 설명변수로 목표변수 예측 / rf)잘 교육을 받았는지 시험을 친다
target_train_predict.shape #target_train.shape 동일

In [ ]:
from sklearn.metrics import accuracy_score # 정확도

In [ ]:
accuracy_score(target_train, target_train_predict) ## 실제값과 교육한 값과 비교(교내 시험)

# 0.696078431372549 정규화 이전 

In [ ]:
target_test_predict = model.predict(features_test) # test한 값의 예측값
target_test_predict.shape #target_test.shape과 동일

In [ ]:
accuracy_score(target_test, target_test_predict)
## 실제test값과 시험값의 예측의 비교(교외 시험)

# 0.6703910614525139 정규화 이전 

#### 추가평가 도구 
- MSE (mean_squared_error, 에러율)
    - 회귀 모델의 성능을 평가하는데 사용되며, 예측 값과 실제 값 사이의 오차를 평가하는 도구
    - 작을수록 모델의 예측이 실제 값과 가깝다고 판단, MSE가 작으면 모델이 데이터를 잘 설명하고 있음을 나타내며, 더 나은 회귀 모델일 가능성이 높음

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mean_squared_error(target_train, target_train_predict) # train에 대한 평가

In [ ]:
## root mean_squared_error(RMSE)
mean_squared_error(target_train, target_train_predict)**0.5

In [ ]:
##mean_absolute_error(MAE)
mean_absolute_error(target_train, target_train_predict) # train에 대한 평가

#### F1
- shape에서 갯수가 같아야 report를 쓸수 있음 
- precision : 정밀도 
- recall : 재현율

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(target_train, target_train_predict)) #교육

## support ; record의 갯수 
## recall ; 한쪽으로 값이 치우쳐져 있음 --> 죽은 데이터가 많다.

In [ ]:
print(classification_report(target_test, target_test_predict)) #시험

### 오차 행렬 (confusion_matrix)
- precision : 정밀도 
- recall : 재현율

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
confusion_matrix(target_train, target_train_predict)

In [ ]:
precision_score(target_train, target_train_predict)

In [ ]:
recall_score(target_train, target_train_predict)

In [ ]:
f1_score(target_train, target_train_predict)